In [7]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

from functions import set_env_variables, instantiate_llm

set_env_variables()

It's all right!


In [8]:
llm = instantiate_llm(model_name="gpt-3.5-turbo")

In [9]:
with open('../section-02/files/winston_churcill_speech.txt', encoding='utf-8') as f:
    text = f.read()

llm.get_num_tokens(text)

4910

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

len(chunks)


3

In [11]:
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=False
)

output = chain.run(chunks)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-va3LEBvnLYt8QWkUrWq6m1TC on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


In [12]:
print(output)

This passage describes the successful evacuation of British and French troops from the beaches of France and Belgium during World War II, with the assistance of the Royal Air Force and Navy. The author emphasizes the need for defense against potential invasion and the determination to continue the fight against tyranny. The passage also mentions the losses suffered by the British Army and the measures being taken to strengthen defenses.


This passage describes the successful evacuation of British and French troops from the beaches of France and Belgium during World War II, with the assistance of the Royal Air Force and Navy. The author emphasizes the need for defense against potential invasion and the determination to continue the fight against tyranny. The passage also mentions the losses suffered by the British Army and the measures being taken to strengthen defenses.